In [1]:
pip install fastparquet 

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.1 MB/s eta 0:00:00:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
# from lightgbm import LGBMClassifier
import tensorflow as tf
from tensorflow import keras

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Nov-2022/sample_submission.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Nov-2022/train_labels.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

## Reading data-files
submission = pd.read_csv(file_content_stream_1)
y_true = pd.read_csv(file_content_stream_2)
df = pd.read_parquet('s3://analytics-data-science-competitions/Tabular-Playground-Series/Tabular-Playground-Nov-2022/preds_logit_concat_gzip.parquet', engine = 'fastparquet')

preds = pd.merge(df, y_true, on = 'id', how = 'left')
preds.head()

,id,0.7301891713.csv,0.6750726968.csv,0.7194704070.csv,0.7107007521.csv,0.6952032365.csv,0.7311830751.csv,0.6736005999.csv,0.7205109360.csv,0.7298116981.csv,0.6737624943.csv,0.7105097012.csv,0.7163693161.csv,0.7225567087.csv,0.6872126167.csv,0.7292947020.csv,0.6798503735.csv,0.7130672940.csv,0.6709092823.csv,0.7391105353.csv,0.6616334858.csv,0.6897190987.csv,0.6963558296.csv,0.7170726761.csv,0.7156669797.csv,0.6814483512.csv,0.7167410246.csv,0.6981892492.csv,0.7119194656.csv,0.6932545022.csv,0.6593521643.csv,0.6743157708.csv,0.7125579799.csv,0.7069356577.csv,0.6884713711.csv,0.6702128703.csv,0.6564147356.csv,0.6977737011.csv,0.6638768547.csv,0.7032756348.csv,0.6862982027.csv,0.6864219834.csv,0.7088904051.csv,0.7212180410.csv,0.7421980731.csv,0.6825341422.csv,0.6848737762.csv,0.7144978161.csv,0.6658856476.csv,0.7195078907.csv,...,0.6870791547.csv,0.6967096004.csv,0.7060967827.csv,0.7113625862.csv,0.6694947166.csv,0.6900693308.csv,0.7128591265.csv,0.6818718747.csv,0.6858034178.csv,0.7122377012.csv,0.7144335192.csv,0.7083170570.csv,0.7142499638.csv,0.7136242881.csv,0.6888742488.csv,0.7087145574.csv,0.6566530931.csv,0.7197358544.csv,0.7101817035.csv,0.7520219713.csv,0.7215952204.csv,0.6795805487.csv,0.7083706040.csv,0.6941218921.csv,0.6926641668.csv,0.6869857003.csv,0.7198785912.csv,0.7287761089.csv,0.7035686386.csv,0.7336496744.csv,0.6872533788.csv,0.7005346672.csv,0.7369069391.csv,0.7156537235.csv,0.7121002711.csv,0.7289228406.csv,0.7174789519.csv,0.7032181627.csv,0.6668147232.csv,0.7345858839.csv,0.7056743178.csv,0.6850481924.csv,0.7359858591.csv,0.7068431477.csv,0.7121179915.csv,0.6613117872.csv,0.6843637618.csv,0.6781392004.csv,0.7222809303.csv,label
0,0,0.913897,2.142608,1.502359,1.453620,1.336538,1.599475,1.236069,1.054479,1.131611,0.984009,1.171605,1.207453,1.400562,1.156730,1.655767,0.405465,1.194848,1.542418,1.921342,0.641228,1.025477,1.378848,1.257294,1.280277,1.467210,1.443668,1.391678,1.419787,1.054552,3.539317,0.983944,1.476214,1.234424,1.307563,2.385436,2.001724,1.257358,1.689581,1.290093,1.277571,1.269417,0.953358,0.838936,1.437067,1.378712,-0.090514,1.220395,2.258836,1.228620,...,1.172375,1.109329,1.245783,1.377996,1.435160,1.195812,1.595924,1.338426,1.170736,1.263989,0.925031,0.862942,1.198344,1.414648,1.305720,0.937820,0.674106,1.475712,1.064896,0.678348,1.330522,1.242018,1.256842,1.234825,1.252730,1.259111,1.237227,1.548106,1.496501,1.588877,1.322937,1.301587,1.143404,1.285821,1.362650,1.133906,1.368576,1.159616,2.507343,0.925301,1.451954,1.199074,1.364975,1.650185,1.184054,0.660613,1.385376,1.596353,1.544845,0.0
1,1,0.678169,0.576310,0.298877,0.301982,0.698756,1.131453,0.569003,0.426954,0.510710,0.430144,0.221108,0.264951,1.556663,1.082496,1.407428,1.152680,0.920520,0.907372,1.113602,0.179283,1.125816,0.637407,0.381426,0.376108,1.140530,0.507529,0.556460,0.232753,0.004920,0.810242,1.154435,0.464158,0.571005,0.498050,0.208698,0.693851,0.753772,0.798899,0.556192,0.626027,0.599505,0.451303,0.421232,1.578557,1.518368,0.012512,0.332251,0.617989,0.828814,...,0.323980,0.483191,0.284638,0.408454,0.674884,0.355031,0.396140,1.250862,0.718760,0.147479,0.554945,0.257595,0.372066,0.811467,0.535556,0.407907,0.046973,0.404249,0.377877,0.950909,0.288525,0.595133,0.411427,0.758979,0.378051,0.539336,0.649062,0.266136,0.484547,0.557583,0.529086,1.068429,0.249275,0.325167,0.454034,0.363870,0.201669,0.191891,0.913241,0.593457,0.576423,0.604565,0.851182,0.545539,0.652546,0.147133,0.615547,0.412286,0.128416,1.0
2,2,1.222971,1.493293,1.997012,1.580462,1.181609,1.527942,1.655077,1.549787,1.668255,1.272369,1.688099,1.579537,-0.373970,1.167650,1.493627,1.152680,1.961822,1.120913,1.743965,0.915126,1.470307,1.403919,1.821767,1.689505,1.227045,1.376995,1.348107,1.619013,0.584926,1.492138,1.441973,1.098612,1.537328,1.360002,1.502614,1.400423,1.176323,1.449841,1.581607,1.417900,1.377263,1.590156,1.667259,1.592731,1.206471,0.565491,1.875071,1.503783,1.565373,...,1.275068,1.503897,1.652910,1.629185,1.435720,1.459440,1.472051,1.205913,1.217617,

In [3]:
train = preds[preds['label'].notnull()]
train['label'] = train['label'].astype(int)

test = preds[preds['label'].isnull()] 

/tmp/ipykernel_10277/3277270779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['label'] = train['label'].astype(int)


In [4]:
lasso_scores = pd.read_csv('lasso_scores_logit.csv')
lasso_scores.head()

,Feature,Coef,abs_coef
0,0.6564517315.csv,2.601288,2.601288
1,0.7183933428.csv,-2.560396,2.560396
2,0.6570599977.csv,2.443219,2.443219
3,0.6454375051.csv,2.201914,2.201914
4,0.7103145610.csv,2.114975,2.114975


In [37]:
to_select = lasso_scores['Feature'][0:50]
to_select.values

array(['0.6564517315.csv', '0.7183933428.csv', '0.6570599977.csv',
       '0.6454375051.csv', '0.7103145610.csv', '0.6495843938.csv',
       '0.6802400070.csv', '0.7120881745.csv', '0.6708990553.csv',
       '0.6911116549.csv', '0.6575661514.csv', '0.6982536185.csv',
       '0.6988389476.csv', '0.7123959613.csv', '0.6603097589.csv',
       '0.6666609229.csv', '0.6601920939.csv', '0.7204986049.csv',
       '0.7113211865.csv', '0.6492627054.csv', '0.7203099744.csv',
       '0.7298182840.csv', '0.6541278746.csv', '0.6657348857.csv',
       '0.6917509910.csv', '0.7215499173.csv', '0.6893289917.csv',
       '0.6507043652.csv', '0.7145930700.csv', '0.7410778621.csv',
       '0.6856099530.csv', '0.6598940198.csv', '0.6914345720.csv',
       '0.7121114883.csv', '0.7086765052.csv', '0.7136612407.csv',
       '0.7163657677.csv', '0.6891839618.csv', '0.6552358052.csv',
       '0.6839403095.csv', '0.7143591677.csv', '0.6785443179.csv',
       '0.7081765641.csv', '0.7168899938.csv', '0.7170741747.c

In [48]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import optimizers

scaler = MinMaxScaler()

X = train[to_select.values]
Y = train['label']

test_new = test[to_select.values]
test_new = pd.DataFrame(scaler.fit_transform(test_new), columns = test_new.columns)

## Defining model 
lgb_md = tf.keras.models.Sequential([
         tf.keras.layers.Dense(32, input_dim = 50, activation = 'relu'),
         tf.keras.layers.Dense(32, activation = 'relu'),
         tf.keras.layers.Dense(32, activation = 'relu'),
         tf.keras.layers.Dense(2, activation = 'softmax')
])

lgb_md.compile(optimizer = 'adam', 
               loss = 'binary_crossentropy')

# lgb_md.compile(optimizer = optimizers.RMSprop(learning_rate = 0.001), 
#                loss = 'binary_crossentropy')

## Defining list to store results
lgb_results = list()
test_preds_lgb_fold_1 = list() 
test_preds_lgb_fold_2 = list()
test_preds_lgb_fold_3 = list()
test_preds_lgb_fold_4 = list()
test_preds_lgb_fold_5 = list()

fold = 1
kfold = StratifiedKFold(n_splits = 5, shuffle = True)
        
for train_ix, test_ix in kfold.split(X, Y):
    
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]

    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
    X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

    ## Building model
    lgb_md.fit(X_train, tf.keras.utils.to_categorical(Y_train, num_classes = 2), 
               verbose = 1, epochs = 30, batch_size = 32, 
               validation_data = (X_test, 
                                  tf.keras.utils.to_categorical(Y_test, num_classes = 2)))
    
    
#     lgb_md = LGBMClassifier(n_estimators = 1000, 
#                             learning_rate = 0.01,
#                             num_leaves = 50,
#                             max_depth = 17, 
#                             lambda_l1 = 3, 
#                             lambda_l2 = 1, 
#                             bagging_fraction = 0.8, 
#                             feature_fraction = 0.8).fit(X_train, Y_train)
        
    ## Predicting on test
    lgb_pred = lgb_md.predict(X_test)[:, 1].astype('float64')
    score = log_loss(Y_test, lgb_pred)
    lgb_results.append(score)
        
    print('Fold ', str(fold), ' result is:', score, '\n')
    
    if (fold == 1):
        test_preds_lgb_fold_1.append(lgb_md.predict(test_new)[:, 1].astype('float64'))
        
    if (fold == 2):
        test_preds_lgb_fold_2.append(lgb_md.predict(test_new)[:, 1].astype('float64'))
        
    if (fold == 3):
        test_preds_lgb_fold_3.append(lgb_md.predict(test_new)[:, 1].astype('float64'))
        
    if (fold == 4):
        test_preds_lgb_fold_4.append(lgb_md.predict(test_new)[:, 1].astype('float64'))
        
    if (fold == 5):
        test_preds_lgb_fold_5.append(lgb_md.predict(test_new)[:, 1].astype('float64'))
    
    fold +=1

print('The average log-loss over 5-fold CV is', np.mean(lgb_results))

Epoch 1/30
500/500 [==============================] - 1s 2ms/step - loss: 0.5745 - val_loss: 0.7201
Epoch 2/30
500/500 [==============================] - 1s 1ms/step - loss: 0.5389 - val_loss: 0.5934
Epoch 3/30
500/500 [==============================] - 1s 2ms/step - loss: 0.5401 - val_loss: 0.5978
Epoch 4/30
500/500 [==============================] - 1s 2ms/step - loss: 0.5312 - val_loss: 0.6151
Epoch 5/30
500/500 [==============================] - 1s 2ms/step - loss: 0.5340 - val_loss: 0.5946
Epoch 6/30
500/500 [==============================] - 1s 2ms/step - loss: 0.5323 - val_loss: 0.6442
Epoch 7/30
500/500 [==============================] - 1s 1ms/step - loss: 0.5325 - val_loss: 0.6144
Epoch 8/30
500/500 [==============================] - 1s 1ms/step - loss: 0.5312 - val_loss: 0.5906
Epoch 9/30
500/500 [==============================] - 1s 1ms/step - loss: 0.5303 - val_loss: 0.7546
Epoch 10/30
500/500 [==============================] - 1s 1ms/step - loss: 0.5341 - val_loss: 0.5707

In [44]:
test_preds_lgb_fold_4[0]

array([0.64723808, 0.77032477, 0.14611351, ..., 0.60055375, 0.54604065,
       0.24457301])

In [39]:
w1 = 1/ lgb_results[0]
w2 = 1/ lgb_results[1]
w3 = 1/ lgb_results[2]
w4 = 1/ lgb_results[3]
w5 = 1/ lgb_results[4]
w_tot = w1 + w2 + w3+ w4 + w5
w1 = w1 / w_tot
w2 = w2 / w_tot
w3 = w3 / w_tot
w4 = w4 / w_tot
w5 = w5 / w_tot

pred1 = w1*test_preds_lgb_fold_1[0]
pred2 = w2*test_preds_lgb_fold_2[0]
pred3 = w3*test_preds_lgb_fold_3[0]
pred4 = w4*test_preds_lgb_fold_4[0]
pred5 = w4*test_preds_lgb_fold_5[0]

In [46]:
submission['pred'] = test_preds_lgb_fold_4[0]
submission.head()

,id,pred
0,20000,0.647238
1,20001,0.770325
2,20002,0.146114
3,20003,0.511132
4,20004,0.006267


In [40]:
submission['pred'] = pred1 + pred2 + pred3 + pred4 + pred5
submission.head()

,id,pred
0,20000,0.624641
1,20001,0.791941
2,20002,0.099367
3,20003,0.488424
4,20004,0.014468


In [47]:
submission.to_csv('submission_TF_lasso_50_fold4.csv', index = False)